In [12]:
# Phase 2: NBA Data Cleaning & Integration
# Purpose: Clean raw data, merge tables, create master dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("="*70)
print("PHASE 2: NBA DATA CLEANING & INTEGRATION")
print("="*70)
print("\n✅ Libraries loaded!")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# Define paths
DATA_DIR = Path('data/raw')
PROCESSED_DIR = Path('data/processed')
PROCESSED_DIR.mkdir(exist_ok=True, parents=True)

print("📂 Loading raw data...")
print("="*70)

# Load main tables
games = pd.read_csv(DATA_DIR / 'games.csv')
games_details = pd.read_csv(DATA_DIR / 'games_details.csv')
teams = pd.read_csv(DATA_DIR / 'teams.csv')
ranking = pd.read_csv(DATA_DIR / 'ranking.csv')
seasons_stats = pd.read_csv(DATA_DIR / 'Seasons_Stats.csv')

# Load betting data
betting_moneyline = pd.read_csv(DATA_DIR / 'nba_betting_money_line.csv')
betting_spread = pd.read_csv(DATA_DIR / 'nba_betting_spread.csv')
betting_totals = pd.read_csv(DATA_DIR / 'nba_betting_totals.csv')
nba_games_all = pd.read_csv(DATA_DIR / 'nba_games_all.csv')

print(f"✅ games: {len(games):,} rows")
print(f"✅ games_details: {len(games_details):,} rows")
print(f"✅ teams: {len(teams):,} rows")
print(f"✅ ranking: {len(ranking):,} rows")
print(f"✅ seasons_stats: {len(seasons_stats):,} rows")
print(f"✅ betting_moneyline: {len(betting_moneyline):,} rows")
print(f"✅ betting_spread: {len(betting_spread):,} rows")
print(f"✅ betting_totals: {len(betting_totals):,} rows")
print(f"✅ nba_games_all: {len(nba_games_all):,} rows")

print(f"\n📊 Total rows loaded: {len(games) + len(games_details) + len(betting_moneyline):,}")


# STEP 1: Remove games with missing scores (postponed/canceled)
print("\n🧹 STEP 1: Cleaning games table")
print("="*70)

print(f"\nBefore cleaning: {len(games):,} games")

# Identify incomplete games
incomplete = games[games['PTS_home'].isna() | games['PTS_away'].isna()]
print(f"Found {len(incomplete)} incomplete games (missing scores)")

if len(incomplete) > 0:
    print("\nIncomplete games (showing first 5):")
    display(incomplete[['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 
                        'VISITOR_TEAM_ID', 'PTS_home', 'PTS_away']].head())
    
    # Remove them
    games_clean = games.dropna(subset=['PTS_home', 'PTS_away'])
    print(f"\n✅ Removed {len(incomplete)} incomplete games")
else:
    games_clean = games.copy()

print(f"After removing incomplete: {len(games_clean):,} games")


# STEP 2: Remove duplicate GAME_IDs
print("\n🧹 STEP 2: Removing duplicate games")
print("="*70)

# Find duplicates
duplicates = games_clean[games_clean.duplicated(subset=['GAME_ID'], keep=False)]
print(f"\nFound {len(duplicates)} duplicate records ({duplicates['GAME_ID'].nunique()} unique games)")

if len(duplicates) > 0:
    print("\nExample duplicates:")
    sample_dup_id = duplicates['GAME_ID'].iloc[0]
    display(games_clean[games_clean['GAME_ID'] == sample_dup_id])
    
    # Keep first occurrence, drop rest
    games_clean = games_clean.drop_duplicates(subset=['GAME_ID'], keep='first')
    print(f"\n✅ Kept first occurrence of each duplicate")

print(f"After removing duplicates: {len(games_clean):,} games")



# STEP 3: Investigate unusual scores
print("\n🔍 STEP 3: Investigating unusual scores")
print("="*70)

# Find abnormally low scores
low_scoring = games_clean[(games_clean['PTS_home'] < 50) | (games_clean['PTS_away'] < 50)]

if len(low_scoring) > 0:
    print(f"\n⚠️  Found {len(low_scoring)} games with scores below 50 points:")
    display(low_scoring[['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 
                          'PTS_home', 'PTS_away', 'SEASON']])
    
    # Check if these are from very old seasons (different rules)
    print("\nAnalysis:")
    for idx, row in low_scoring.iterrows():
        total = row['PTS_home'] + row['PTS_away']
        print(f"  Season {row['SEASON']}: Total score = {total} points")
        if int(row['SEASON']) < 1954:  # Before shot clock era
            print("    → Pre-shot-clock era game (valid low score)")
        else:
            print("    ⚠️  Unusual for modern NBA - possible error")
    
    # Decision: Keep if pre-1954, investigate if after
    # For now, we'll keep all (they passed our >33 threshold)
    print("\n✅ Keeping all games (above minimum threshold)")
else:
    print("✅ No unusually low-scoring games found")

print(f"\nFinal games count: {len(games_clean):,}")


# STEP 4: Standardize date formats
print("\n📅 STEP 4: Standardizing date format")
print("="*70)

# Convert to datetime
games_clean['GAME_DATE'] = pd.to_datetime(games_clean['GAME_DATE_EST'])
games_clean['YEAR'] = games_clean['GAME_DATE'].dt.year
games_clean['MONTH'] = games_clean['GAME_DATE'].dt.month
games_clean['DAY_OF_WEEK'] = games_clean['GAME_DATE'].dt.dayofweek  # 0=Monday, 6=Sunday

print("✅ Date columns created:")
print(f"   - GAME_DATE (datetime)")
print(f"   - YEAR (integer)")
print(f"   - MONTH (integer)")
print(f"   - DAY_OF_WEEK (0=Mon, 6=Sun)")

# Show date range
print(f"\n📊 Date range: {games_clean['GAME_DATE'].min().date()} to {games_clean['GAME_DATE'].max().date()}")
print(f"   Seasons covered: {games_clean['SEASON'].min()} to {games_clean['SEASON'].max()}")

# Games per year
games_per_year = games_clean.groupby('YEAR').size()
print(f"\n📈 Games per year:")
print(games_per_year.tail(10))


# STEP 5: Handle DNP (Did Not Play) players
print("\n🧹 STEP 5: Processing player participation data")
print("="*70)

# Create 'played' flag
games_details['PLAYED'] = (
    games_details['MIN'].notna() & 
    (games_details['MIN'] != '0:00') & 
    (games_details['MIN'] != '0')
)

dnp_count = (~games_details['PLAYED']).sum()
played_count = games_details['PLAYED'].sum()

print(f"Players who played: {played_count:,} ({played_count/len(games_details)*100:.1f}%)")
print(f"Players who DNP: {dnp_count:,} ({dnp_count/len(games_details)*100:.1f}%)")

print("\n💡 Strategy: KEEP DNP records - they indicate injuries/rest patterns")

# For DNP players, fill stats with 0
stat_cols = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 
             'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS']

print("\n📝 Filling DNP player stats with 0...")
for col in stat_cols:
    if col in games_details.columns:
        games_details[col] = games_details[col].fillna(0)

print("✅ DNP players processed")


# STEP 6: Merge betting odds with games
print("\n💰 STEP 6: Merging betting data with games")
print("="*70)

# We'll use Pinnacle Sports odds (most respected sportsbook)
# Filter to just Pinnacle
pinnacle_ml = betting_moneyline[betting_moneyline['book_name'] == 'Pinnacle Sports'].copy()
pinnacle_spread = betting_spread[betting_spread['book_name'] == 'Pinnacle Sports'].copy()
pinnacle_totals = betting_totals[betting_totals['book_name'] == 'Pinnacle Sports'].copy()

print(f"Pinnacle moneyline: {len(pinnacle_ml):,} rows")
print(f"Pinnacle spread: {len(pinnacle_spread):,} rows")
print(f"Pinnacle totals: {len(pinnacle_totals):,} rows")

# Rename columns for clarity
pinnacle_ml = pinnacle_ml.rename(columns={
    'game_id': 'GAME_ID',
    'price1': 'HOME_MONEYLINE',
    'price2': 'AWAY_MONEYLINE'
})

pinnacle_spread = pinnacle_spread.rename(columns={
    'game_id': 'GAME_ID',
    'spread1': 'HOME_SPREAD',
    'spread2': 'AWAY_SPREAD'
})

pinnacle_totals = pinnacle_totals.rename(columns={
    'game_id': 'GAME_ID',
    'total1': 'TOTAL_LINE'
})

# Keep only needed columns
pinnacle_ml = pinnacle_ml[['GAME_ID', 'HOME_MONEYLINE', 'AWAY_MONEYLINE']]
pinnacle_spread = pinnacle_spread[['GAME_ID', 'HOME_SPREAD', 'AWAY_SPREAD']]
pinnacle_totals = pinnacle_totals[['GAME_ID', 'TOTAL_LINE']]

# Remove duplicates (some games have multiple lines)
pinnacle_ml = pinnacle_ml.drop_duplicates(subset=['GAME_ID'], keep='last')  # Keep closing line
pinnacle_spread = pinnacle_spread.drop_duplicates(subset=['GAME_ID'], keep='last')
pinnacle_totals = pinnacle_totals.drop_duplicates(subset=['GAME_ID'], keep='last')

print("\n✅ Betting data prepared (closing lines)")


# STEP 7: Merge everything into master dataset
print("\n🔗 STEP 7: Creating master dataset")
print("="*70)

# Start with cleaned games
master = games_clean.copy()
print(f"Starting with: {len(master):,} games")

# Merge betting data
print("\nMerging betting odds...")
master = master.merge(pinnacle_ml, on='GAME_ID', how='left')
print(f"  After moneyline: {len(master):,} games")

master = master.merge(pinnacle_spread, on='GAME_ID', how='left')
print(f"  After spread: {len(master):,} games")

master = master.merge(pinnacle_totals, on='GAME_ID', how='left')
print(f"  After totals: {len(master):,} games")

# Check how many games have betting data
has_ml = master['HOME_MONEYLINE'].notna().sum()
has_spread = master['HOME_SPREAD'].notna().sum()
has_total = master['TOTAL_LINE'].notna().sum()

print(f"\n📊 Betting data coverage:")
print(f"  Games with moneyline: {has_ml:,} ({has_ml/len(master)*100:.1f}%)")
print(f"  Games with spread: {has_spread:,} ({has_spread/len(master)*100:.1f}%)")
print(f"  Games with totals: {has_total:,} ({has_total/len(master)*100:.1f}%)")

# Show sample
print("\n🔍 Sample of master dataset:")
display(master[['GAME_DATE', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'PTS_home', 
                'PTS_away', 'HOME_MONEYLINE', 'HOME_SPREAD', 'TOTAL_LINE']].head(3))


# STEP 8: Calculate betting outcomes (did bets win?)
print("\n🎯 STEP 8: Calculating betting outcomes")
print("="*70)

# 1. Moneyline outcome (did home team win?)
master['ML_HOME_WIN'] = master['HOME_TEAM_WINS']  # Already have this
master['ML_AWAY_WIN'] = 1 - master['HOME_TEAM_WINS']

# 2. Spread outcome (did home team cover?)
master['ACTUAL_MARGIN'] = master['PTS_home'] - master['PTS_away']
master['SPREAD_HOME_COVER'] = (
    master['ACTUAL_MARGIN'] > -master['HOME_SPREAD']
).astype(float)

# 3. Totals outcome (did total go over?)
master['ACTUAL_TOTAL'] = master['PTS_home'] + master['PTS_away']
master['TOTAL_OVER'] = (
    master['ACTUAL_TOTAL'] > master['TOTAL_LINE']
).astype(float)

# Show examples
print("✅ Betting outcomes calculated:\n")
print("Example games:")
sample = master[master['HOME_SPREAD'].notna()].head(3)
display(sample[['GAME_DATE', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 
                'PTS_home', 'PTS_away', 'ACTUAL_MARGIN',
                'HOME_SPREAD', 'SPREAD_HOME_COVER']])

# Summary statistics
if master['HOME_SPREAD'].notna().sum() > 0:
    cover_rate = master['SPREAD_HOME_COVER'].mean() * 100
    over_rate = master['TOTAL_OVER'].mean() * 100
    
    print(f"\n📊 Historical outcomes:")
    print(f"  Home team covers spread: {cover_rate:.1f}%")
    print(f"  Totals go OVER: {over_rate:.1f}%")
    print(f"  Home team wins outright: {master['ML_HOME_WIN'].mean()*100:.1f}%")


# STEP 9: Convert American odds to implied probabilities
print("\n🧮 STEP 9: Converting odds to probabilities")
print("="*70)

def american_to_prob(odds):
    """Convert American odds to implied probability"""
    if pd.isna(odds):
        return np.nan
    if odds > 0:
        return 100 / (odds + 100)
    else:
        return -odds / (-odds + 100)

# Convert moneyline odds
master['HOME_WIN_PROB'] = master['HOME_MONEYLINE'].apply(american_to_prob)
master['AWAY_WIN_PROB'] = master['AWAY_MONEYLINE'].apply(american_to_prob)

print("✅ Converted odds to probabilities")
print("\nExample conversions:")
sample = master[master['HOME_MONEYLINE'].notna()].head(3)
display(sample[['GAME_DATE', 'HOME_TEAM_ID', 'HOME_MONEYLINE', 
                'HOME_WIN_PROB', 'HOME_TEAM_WINS']])

print("\n💡 Interpretation:")
print("  - Negative odds (e.g., -150) = favorite")
print("  - Positive odds (e.g., +130) = underdog")
print("  - Probability > 0.5 = expected to win")


# STEP 10: Add team names for readability
print("\n🏀 STEP 10: Adding team names")
print("="*70)

# Create team lookup
team_lookup = teams.set_index('TEAM_ID')['ABBREVIATION'].to_dict()

# Add team abbreviations
master['HOME_TEAM'] = master['HOME_TEAM_ID'].map(team_lookup)
master['AWAY_TEAM'] = master['VISITOR_TEAM_ID'].map(team_lookup)

# Create matchup string
master['MATCHUP'] = master['AWAY_TEAM'] + ' @ ' + master['HOME_TEAM']

print("✅ Team names added")
print("\nSample matchups:")
print(master[['GAME_DATE', 'MATCHUP', 'PTS_away', 'PTS_home']].head(5))


# STEP 11: Create two versions - ALL games vs BETTING games only
print("\n📊 STEP 11: Creating filtered datasets")
print("="*70)

# Version 1: ALL cleaned games
master_all = master.copy()
print(f"master_all: {len(master_all):,} games (all cleaned games)")

# Version 2: Only games with complete betting data
master_betting = master[
    master['HOME_MONEYLINE'].notna() & 
    master['HOME_SPREAD'].notna() & 
    master['TOTAL_LINE'].notna()
].copy()

print(f"master_betting: {len(master_betting):,} games (only with complete odds)")

# Show date range for betting dataset
print(f"\nBetting dataset date range:")
print(f"  From: {master_betting['GAME_DATE'].min().date()}")
print(f"  To: {master_betting['GAME_DATE'].max().date()}")

# Games per season in betting dataset
betting_per_season = master_betting.groupby('SEASON').size()
print(f"\nGames with betting data per season:")
print(betting_per_season.tail(10))


# STEP 12: Generate summary statistics
print("\n📈 STEP 12: Summary statistics")
print("="*70)

print("\n🏀 SCORING STATISTICS:")
print(f"  Average home score: {master_all['PTS_home'].mean():.1f}")
print(f"  Average away score: {master_all['PTS_away'].mean():.1f}")
print(f"  Home court advantage: {(master_all['PTS_home'] - master_all['PTS_away']).mean():.1f} points")
print(f"  Home win rate: {master_all['HOME_TEAM_WINS'].mean()*100:.1f}%")

if len(master_betting) > 0:
    print("\n💰 BETTING STATISTICS:")
    print(f"  Average home spread: {master_betting['HOME_SPREAD'].mean():.2f}")
    print(f"  Average total line: {master_betting['TOTAL_LINE'].mean():.1f}")
    print(f"  Home covers spread: {master_betting['SPREAD_HOME_COVER'].mean()*100:.1f}%")
    print(f"  Games go OVER: {master_betting['TOTAL_OVER'].mean()*100:.1f}%")

print("\n📅 TEMPORAL STATISTICS:")
games_by_dow = master_all.groupby('DAY_OF_WEEK').size()
dow_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print("  Games by day of week:")
for dow, count in games_by_dow.items():
    print(f"    {dow_names[dow]}: {count:,}")


# STEP 13: Save cleaned datasets
print("\n💾 STEP 13: Saving cleaned data")
print("="*70)

# Save master datasets
master_all.to_csv(PROCESSED_DIR / 'games_cleaned_all.csv', index=False)
print(f"✅ Saved: games_cleaned_all.csv ({len(master_all):,} games)")

master_betting.to_csv(PROCESSED_DIR / 'games_with_betting.csv', index=False)
print(f"✅ Saved: games_with_betting.csv ({len(master_betting):,} games)")

# Save cleaned games_details
games_details.to_csv(PROCESSED_DIR / 'games_details_cleaned.csv', index=False)
print(f"✅ Saved: games_details_cleaned.csv ({len(games_details):,} records)")

# Save metadata
metadata = {
    'cleaned_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_games': len(master_all),
    'games_with_betting': len(master_betting),
    'date_range_start': str(master_all['GAME_DATE'].min().date()),
    'date_range_end': str(master_all['GAME_DATE'].max().date()),
    'seasons_covered': f"{master_all['SEASON'].min()}-{master_all['SEASON'].max()}"
}

import json
with open(PROCESSED_DIR / 'cleaning_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Saved: cleaning_metadata.json")

print(f"\n📂 All cleaned files saved to: {PROCESSED_DIR}")


# STEP 14: Generate final data quality report
print("\n" + "="*70)
print("✅ PHASE 2 COMPLETE - DATA CLEANING SUMMARY")
print("="*70)

print("\n📊 BEFORE CLEANING:")
print(f"  Raw games: 26,651")
print(f"  Issues: 99 incomplete, 29 duplicates")

print("\n📊 AFTER CLEANING:")
print(f"  Clean games (all): {len(master_all):,}")
print(f"  Games with betting data: {len(master_betting):,}")
print(f"  Player-game records: {len(games_details):,}")

print("\n📋 CREATED DATASETS:")
print("  1. games_cleaned_all.csv - All cleaned games")
print("  2. games_with_betting.csv - Games with complete betting odds")
print("  3. games_details_cleaned.csv - Player stats with DNP flags")

print("\n🎯 KEY FEATURES CREATED:")
print("  ✅ Betting outcomes (ML_HOME_WIN, SPREAD_HOME_COVER, TOTAL_OVER)")
print("  ✅ Implied probabilities (HOME_WIN_PROB, AWAY_WIN_PROB)")
print("  ✅ Actual margins and totals")
print("  ✅ Time features (YEAR, MONTH, DAY_OF_WEEK)")
print("  ✅ Team abbreviations and matchups")
print("  ✅ DNP player flags")

print("\n📈 DATA QUALITY:")
missing_pct = (master_betting.isnull().sum().sum() / 
               (len(master_betting) * len(master_betting.columns))) * 100
print(f"  Missing values: {missing_pct:.2f}%")
print(f"  Duplicates: 0 (removed)")
print(f"  Date range: {master_betting['GAME_DATE'].min().date()} to {master_betting['GAME_DATE'].max().date()}")

print("\n🎯 READY FOR:")
print("  → Phase 3: Exploratory Data Analysis")
print("  → Phase 4: Feature Engineering")
print("  → Phase 5: Machine Learning Models")

print("\n" + "="*70)
print(f"Cleaning completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)


# Preview the final dataset
print("\n🔍 FINAL DATASET PREVIEW")
print("="*70)

print("\nColumns in master dataset:")
print(master_betting.columns.tolist())

print(f"\nFirst 5 games with complete data:")
display(master_betting[[
    'GAME_DATE', 'MATCHUP', 
    'PTS_home', 'PTS_away', 'ACTUAL_MARGIN',
    'HOME_SPREAD', 'SPREAD_HOME_COVER',
    'TOTAL_LINE', 'ACTUAL_TOTAL', 'TOTAL_OVER',
    'HOME_MONEYLINE', 'HOME_WIN_PROB', 'HOME_TEAM_WINS'
]].head())

print("\n💡 NEXT STEPS:")
print("1. Reply: 'Phase 2 complete!'")
print("2. I'll create Phase 3: Exploratory Data Analysis")
print("   (Visualizations, patterns, insights)")


PHASE 2: NBA DATA CLEANING & INTEGRATION

✅ Libraries loaded!
Start time: 2026-02-14 15:41:31

📂 Loading raw data...
✅ games: 26,651 rows
✅ games_details: 668,628 rows
✅ teams: 30 rows
✅ ranking: 210,342 rows
✅ seasons_stats: 24,691 rows
✅ betting_moneyline: 125,286 rows
✅ betting_spread: 131,690 rows
✅ betting_totals: 131,386 rows
✅ nba_games_all: 125,624 rows

📊 Total rows loaded: 820,565

🧹 STEP 1: Cleaning games table

Before cleaning: 26,651 games
Found 99 incomplete games (missing scores)

Incomplete games (showing first 5):


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,PTS_home,PTS_away
19175,2003-10-24,10300116,1610612753,1610612762,NaN,NaN
19176,2003-10-24,10300108,1610612737,1610612764,NaN,NaN
19177,2003-10-24,10300109,1610612738,1610612751,NaN,NaN
19178,2003-10-24,10300113,1610612759,1610612745,NaN,NaN
19179,2003-10-24,10300112,1610612749,1610612765,NaN,NaN



✅ Removed 99 incomplete games
After removing incomplete: 26,552 games

🧹 STEP 2: Removing duplicate games

Found 58 duplicate records (29 unique games)

Example duplicates:


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
2477,2021-01-01,22000070,Final,1610612765,1610612738,2020,1610612765,96.0,0.409,0.65,0.324,18.0,48.0,1610612738,93.0,0.47,0.5,0.258,20.0,45.0,1
3102,2021-01-01,22000070,Final,1610612765,1610612738,2020,1610612765,96.0,0.409,0.65,0.324,18.0,48.0,1610612738,93.0,0.47,0.5,0.258,20.0,44.0,1



✅ Kept first occurrence of each duplicate
After removing duplicates: 26,523 games

🔍 STEP 3: Investigating unusual scores

⚠️  Found 1 games with scores below 50 points:


...


📊 Historical outcomes:
  Home team covers spread: 35.8%
  Totals go OVER: 27.5%
  Home team wins outright: 58.9%

🧮 STEP 9: Converting odds to probabilities
✅ Converted odds to probabilities

Example conversions:


,GAME_DATE,HOME_TEAM_ID,HOME_MONEYLINE,HOME_WIN_PROB,HOME_TEAM_WINS
4029,2014-06-15,1610612759,200.0,0.333333,1
4030,2014-06-12,1610612748,205.0,0.327869,0
4031,2014-06-10,1610612748,169.0,0.371747,0



💡 Interpretation:
  - Negative odds (e.g., -150) = favorite
  - Positive odds (e.g., +130) = underdog
  - Probability > 0.5 = expected to win

🏀 STEP 10: Adding team names
✅ Team names added

Sample matchups:
   GAME_DATE    MATCHUP  PTS_away  PTS_home
0 2022-12-22  SAS @ NOP     117.0     126.0
1 2022-12-22  WAS @ UTA     112.0     120.0
2 2022-12-21  MIL @ CLE     106.0     114.0
3 2022-12-21  DET @ PHI      93.0     113.0
4 2022-12-21  CHI @ ATL     110.0     108.0

📊 STEP 11: Creating filtered datasets
master_all: 26,523 games (all cleaned games)
master_betting: 14,816 games (only with complete odds)

Betting dataset date range:
  From: 2006-11-01
  To: 2018-06-08

Games with betting data per season:
SEASON
2008    1232
2009    1254
2010    1204
2011    1103
2012    1298
2013    1289
2014    1283
2015    1277
2016    1261
2017    1210
dtype: int64

📈 STEP 12: Summary statistics

🏀 SCORING STATISTICS:
  Average home score: 103.4
  Average away score: 100.6
  Home court advantage: 2

,GAME_DATE,MATCHUP,PTS_home,PTS_away,ACTUAL_MARGIN,HOME_SPREAD,SPREAD_HOME_COVER,TOTAL_LINE,ACTUAL_TOTAL,TOTAL_OVER,HOME_MONEYLINE,HOME_WIN_PROB,HOME_TEAM_WINS
4029,2014-06-15,MIA @ SAS,104.0,87.0,17.0,5.5,1.0,197.5,191.0,0.0,200.0,0.333333,1
4030,2014-06-12,SAS @ MIA,86.0,107.0,-21.0,5.0,0.0,195.0,193.0,0.0,205.0,0.327869,0
4031,2014-06-10,SAS @ MIA,92.0,111.0,-19.0,4.5,0.0,197.0,203.0,1.0,169.0,0.371747,0
4032,2014-06-08,MIA @ SAS,96.0,98.0,-2.0,4.0,1.0,198.5,194.0,0.0,151.0,0.398406,0
4033,2014-06-05,MIA @ SAS,110.0,95.0,15.0,5.5,1.0,198.5,205.0,1.0,189.0,0.346021,1



💡 NEXT STEPS:
1. Reply: 'Phase 2 complete!'
2. I'll create Phase 3: Exploratory Data Analysis
   (Visualizations, patterns, insights)
